In [54]:
import pandas as pd
import numpy as np
from itertools import product
import copy
from tqdm import tqdm

In [55]:
df= pd.DataFrame({
    "N_candidat": ["0001", "0002", "0003", "0004", "0005"],
    "S1":[15, 11, 14, 16, 11],
    "S2": [13, 5, 12, 15, 11],
    "S3":[16, 8, 11, 15, 11],
    "S4": [17, 12, 15, 18, 11],
    "S5": [13, 10, 13, 15, 11],

    "Logique": [1, 1, 1, 0, 0],
    "BDD": [1, 1, 1, 0, 0],
    "PS": [1, 1, 1, 1, 0],
    "Moyenne": [2, 0, 1, 2, 0],
    "Eligible": [True, True, True, False, False],
    "Decision": ['A','R','L','R','R']
})
V = list(df.columns)[1:-1]
display(V, df)

['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS', 'Moyenne', 'Eligible']

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


# ACi
## AC1

In [56]:
#ligne existe, la décision est bonne et toutes les variables de X ont les bonnes valeurs aussi
def AC1(N, X, decision, df=df):
    c = df[df.N_candidat == N].copy()
    return len(c) == 1 and  c.Decision.iloc[0] == decision and all((str(c.loc[:, [a]].iloc[0, 0]) == str(X[a])) for a in X) 


In [57]:
def test_AC1():
    display(df)
    # Cas de succès
    assert AC1("0001", {"Logique": 1}, 'A')
    assert AC1("0001", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    assert AC1("0004", {"S2":15, "BDD":0}, 'R')

    # Cas d'échec
    assert not AC1("xxxx", {"S1":15, "BDD":1, "Moyenne":2, 'Eligible': True}, 'A')
    display("AC1 est bien correcte.")

test_AC1()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


'AC1 est bien correcte.'

## AC2

In [58]:
coeffs= [1, 1, 1, 1, 1] #calcul de la Moyenne
seuils = [12, 12, 12] # Calcul du status d'éligibilité (logique, BDD, PS)
thresholds = [12, 14] # prendre la décision

def med(S1, S2, S3, S4, S5, Logique, BDD, PS, coeffs=coeffs, seuils=seuils, thresholds2=thresholds):
    moyenne = (S1 * coeffs[0] + S2 * coeffs[1] + S3 * coeffs[2] + S4 * coeffs[3] + S5 * coeffs[4]) / sum(coeffs)
    e = (Logique*BDD*PS==1)
    m = (moyenne>thresholds2[1])*2 + (thresholds2[0] <= moyenne <= thresholds2[1])
    if e and m==2:
        d= 'A'
    elif e and m==1:
        d='L'
    else:
        d='R'
    return m, e, d

In [59]:
def AC2(N_candidat, X, decision, df=df):
    candidat=df[df.N_candidat==N_candidat].copy()
    if (len(candidat)==0):
        return False
    vals=candidat.iloc[0].to_dict()
    X_var=list(X.keys())
    
    #W=list(set(candidat.columns)-set(X_var)) #optionnel

    variable_domains = {
        "S1": range(0, 21),
        "S2": range(0, 21),
        "S3": range(0, 21),
        "S4": range(0, 21),
        "S5": range(0, 21),
        "BDD": [0, 1],
        "Logique": [0, 1],
        "PS": [0, 1],
        "Moyenne":[0,1,2],
        "Eligible":[True,False]
    }
    def generate_combinations(X, variable_domains):
        # Générer toutes les combinaisons possibles de valeurs pour les clés de X_var
        combinations = product(*[variable_domains[var] for var in X])

        # Exclure la combinaison identique à X
        return [combination for combination in combinations if combination != tuple(X.values())]

    combinations = generate_combinations(X, variable_domains)
    
    # Vérifier chaque combinaison
    for combination in combinations:
        x_prime = dict(zip(X_var, combination))
        for k in x_prime.keys():
            vals[k]=x_prime[k]
        m,e,d=med(**{k: v for k, v in vals.items() if k in ['S1', 'S2', 'S3', 'S4', 'S5', 'Logique', 'BDD', 'PS']})

        if ('Eligible' in x_prime and x_prime['Eligible']!=e) or ('Moyenne' in x_prime and x_prime['Moyenne']!=m) :
            continue
        if d!=decision:
            #print(f"AC2 est vérifié pour une affectation x_prime: {x_prime}")
            return True
    return False

In [60]:
def test_AC2():
	display(df)
	# Cas de succès
	assert AC2("0001", {"S1": 15},'A')
	assert AC2("0002", {"S1": 13, "S5": 11}, 'R')
	assert AC2("0003", {"Logique": 1, "S5": 11, "S2":8},'L')

	# Cas d'échec
	assert not AC2("0004", {"S5": 11, "BDD": 0},'R')
	assert not AC2("0005", {"S1": 13, "BDD": 0}, 'R')
	print("AC2 est bien correcte")

test_AC2()



,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC2 est bien correcte


## AC3:

In [61]:
from itertools import combinations

def AC3(N, X, decision ,df=df):
    keys = list(X.keys())
    subsets = [dict(zip(combination, (X[key] for key in combination))) for r in range(1, len(keys)) for combination in combinations(keys, r)]
    return all(not AC1(N, subset, decision, df) or not AC2(N, subset, decision) for subset in subsets)


In [62]:
def test_AC3():
	display(df)
	# Cas de succès
	assert AC3("0001", {"Logique": 1}, 'A')
	assert AC3("0004", {"Logique": 0, "BDD": 0}, 'R')
	assert AC3("0004", {"Logique": 0, "S1": 16}, 'R')

	# Cas d'échec
	assert not AC3("0001", {"Logique": 1, "PS": 1}, 'A')
	assert not AC3("0004", {"Logique": 0,  "BDD": 0,"S1": 16}, 'R')
	print("AC3 est bien correcte")

test_AC3()


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


AC3 est bien correcte


In [63]:
def AC(N, X, decision, df=df, afficher = False):
  res1=AC1(N, X, decision, df)
  res2=AC2(N, X, decision, df)
  res3=AC3(N, X, decision, df)
  if afficher:
      print(f"AC1={res1} et AC2={res2} et AC3={res3}")
  return res1, res2, res3

# BCi
## BC1 et BC2

In [64]:
def generate_supersets(N, X, V, df):
	candidat = df[df['N_candidat']==N]
	V_sauf_X=[var for var in V if not var in X.keys()]
	#print(V_sauf_X)
	supersets = [X]
	for var in V_sauf_X:
		valeur = candidat[var].iloc[0]
		copie = copy.deepcopy(supersets)
		for dictio in copie:
			dictio[var]= valeur
		supersets =supersets + copie
	unique_dicts = {tuple(sorted(d.items())) for d in supersets}
	unique_list_of_dicts = [dict(item) for item in unique_dicts]
	return sorted(unique_list_of_dicts, key=lambda x:len(x))

def BC1(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		if all(AC(N, superset, decision[0], df)):
			print(f"La actual cause est: {superset} \n")
			return True
	return False

def BC2(N, X, decision, df=df):
	supersets = generate_supersets(N, X[1], V, df)
	for superset in tqdm(supersets):
		if all(AC(N, superset, decision[1], df)):
			print("La actual cause est: ", superset)
			return True
	return False

Les performances ne sont pas très bonne, en terme de complexité. Nous proposons une version améliorée.

In [91]:
def BC1_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X[0], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision[0], df)
		if not ac1:
			return False
		if ac2 and not ac3:
			return False
		print(f"La actual cause est: {superset} \n")
		return True
	return False

def BC2_beta(N, X, decision, df=df):
	supersets = generate_supersets(N, X[1], V, df)
	for superset in tqdm(supersets):
		#print("\n",superset)
		ac1, ac2, ac3 = AC(N, superset, decision[1], df)
		if not ac1:
			return False
		if ac2 and not ac3:
			return False
		print(f"La actual cause est: {superset} \n")
		return True
	return False

In [87]:
def test_BC1(f=BC1_beta, only_echec = True):
	display(df)
	# Cas de succès
	if not only_echec:
		assert f("0001", ({"S2": 15}, _), ('A', _))
		assert f("0001", ({"Logique": 1}, _), ('A', _)) 
		assert f("0005", ({"Logique": 0, "BDD": 0, "PS": 0}, _), ('R', _))

	# Cas d'échec
	assert not f("0001", ({"Logique": 1, "BDD": 1}, _), ('A', _)) #AC3
	assert not f("0004", ({"Logique": 1, "BDD": 1, "S1": 16}, _), ('R', _)) #AC1 n'est pas vérifié
	assert not f("0004", ({"Logique": 0, "BDD": 0, "S1": 16}, _), ('R', _)) # car S1 n'appartent pas à la actual cause
	print("Les tests pour BC1 sont terminés.")

In [40]:
test_BC1(f=BC1)
# Nous avons arreté l'éxécution du code


,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  2%|▏         | 12/512 [00:00<00:04, 115.22it/s]

 78%|███████▊  | 399/512 [03:55<01:06,  1.69it/s]


KeyboardInterrupt: 

In [88]:
test_BC1()

,N_candidat,S1,S2,S3,S4,S5,Logique,BDD,PS,Moyenne,Eligible,Decision
0,0001,15,13,16,17,13,1,1,1,2,True,A
1,0002,11,5,8,12,10,1,1,1,0,True,R
2,0003,14,12,11,15,13,1,1,1,1,True,L
3,0004,16,15,15,18,15,0,0,1,2,False,R
4,0005,11,11,11,11,11,0,0,0,0,False,R


  0%|          | 0/128 [00:00<?, ?it/s]

Les tests pour BC1 sont terminés.


## BC3

In [89]:
def BC3(X, df=None):
    X1, X2 = X
    
    # Check si les deux X1 et X2 contiennent les memes variables
    if set(X1.keys()) != set(X2.keys()):
        return False
    
    # Check que l'intersection des deux affectations est vide
    for key in X1.keys():
        if X1[key] == X2[key]:
            return False
    
    return True

In [90]:
def test_BC3():
    # Cas de succès
    assert BC3(({"S2":12, "BDD":0}, {"S2":11, "BDD":1}))

    # Cas d'échec
    assert not BC3(({"S1":15, "BDD":1, "Moyenne":2, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))
    assert not BC3(({"S1":15, "BDD":1, 'Eligible': False}, {"S1":15, "BDD":0, "Moyenne":1, 'Eligible': True}))

    print("BC3 est bien correcte.")

test_BC3()


BC3 est bien correcte.


## BC4 